In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
# Import necessary librarys
import tensorflow as tf
from tensorflow import keras
from keras.layers import Layer, Input, Dense, Concatenate
from keras.initializers import RandomUniform, Zeros
from sklearn.model_selection import train_test_split
import keras_tuner as kt
import numpy as np



2024-03-25 15:51:25.268452: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-25 15:51:25.268576: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-25 15:51:25.419283: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [17]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '0'  # Show all logs, including info (default is '0')
import tensorflow as tf
tf.get_logger().setLevel('INFO')


In [3]:
print(tf.__version__)


2.15.0


In [15]:
class TruncateLayer(Layer):
    """
    A Keras layer that truncates every input to a specified range.

    Parameters
    ----------
    beta : float
        The value used to determine the truncation range.
        The inputs will be clipped to the range [-beta, beta].

    Methods
    -------
    call(inputs)
        Applies truncation to the inputs based on the `beta` value.

    """

    def __init__(self, beta):
        super().__init__()
        self.beta = tf.constant(beta, dtype=tf.float32)

    def call(self, inputs):
        return tf.clip_by_value(
            inputs, clip_value_min=-self.beta, clip_value_max=self.beta
        )


class L1Projection(keras.constraints.Constraint):
    """
    A Keras weight constraint that projects the weight vector w on a vector
    with L1-norm smaller or equal to gamma.

    Parameters
    ----------
    gamma: float
        The value to constraint the L1-norm.
        The weights will be projected on a feasible set where the L1-norm of
        the weight vector is smaller or equal to gamma.

    Methods
    -------
    projection_l1(w)
        Applies L1-projection to the weight vector w based on the 'gamma' value.

    """

    def __init__(self, gamma):
        super().__init__()
        # Error handling:
        if gamma is None:
            raise ValueError("Missing required argument gamma.")
        self.gamma = tf.constant(gamma, dtype=tf.float32)

    def __call__(self, w):
        return self.apply_l1_projection(w)

    @tf.function
    def apply_l1_projection(self, w):
        if tf.norm(w, ord=1) <= self.gamma:
            return w
        # Apply L1-projection on weight vector w
        abs_w = tf.abs(w)

        # Compute cumulative sum of the sorted absolute weights
        u = tf.sort(abs_w, direction="DESCENDING")
        svp = tf.cumsum(u)

        # Find the position where the condition is violated for the first time
        cond = tf.cast(svp - self.gamma, tf.float64) / tf.range(
            1, tf.size(u) + 1, dtype=tf.float64
        )
        k = tf.reduce_max(tf.where(tf.cast(u, tf.float64) > cond))

        # Compute the threshold value
        theta = tf.cast(tf.gather(svp, k) - self.gamma, tf.float32) / tf.cast(
            k + 1, tf.float32
        )

        return tf.math.sign(w) * tf.maximum(abs_w - theta, 0)

    def get_config(self):
        return {"gamma": self.gamma}


class L2ProjectionModel(keras.Model):
    """
    A Keras model that performs an L2-projection of the weights in every
    training step, to ensure that the eucledian distance between the weights and
    the initial weights is smaller or equal to 'delta'.

    Parameters
    ----------
    delta: float
        The value to specify radius of the L2-ball around 'init_vars'.
        The weights will be projected on the feasible set where the L2-norm of
        the difference between current weight vector and the initial weight
        vector is smaller or equal to delta.

    """

    def __init__(
        self,
        delta=1,
        sub_networks=None,
        output_layer=None,
        num_networks=None,
        num_layers=None,
        num_neurons=None,
        beta=None,
        gamma=None,
        **kwargs
    ):
        super().__init__(**kwargs)
        self.delta = tf.constant(delta, dtype=tf.float32)
        self.init_vars = None

        # Optional parameters for analysis
        self.sub_networks = sub_networks
        self.output_layer = output_layer
        self.num_networks = num_networks
        self.num_layers = num_layers
        self.num_neurons = num_neurons
        self.beta = tf.constant(beta) if beta is not None else None
        self.gamma = tf.constant(gamma) if gamma is not None else None
    
    def train_step(self, data):  # sourcery skip: avoid-builtin-shadow
        """
        Custom training step for the model.

        After applying the gradients, the weight vector will be projected on the
        L2-ball with radius 'delta' around the center 'init_vars'.

        Parameters
        ----------
        data: float
            data used for training

        """

        # Unpack the data used for training
        x, y = data

        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            # Compute the loss value
            loss = self.compute_loss(y=y, y_pred=y_pred)

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))

        # Apply L2 projection
        vars = self.trainable_variables[:-1]
        vars_diff = [v - v0 for v, v0 in zip(vars, self.init_vars)]

        if tf.linalg.global_norm(vars_diff) > self.delta:
            # Project the weights and reshape back
            projected_vars = self.apply_l2_projection(vars_diff)
            for v, pv in zip(vars, projected_vars):
                v.assign(pv)

        # Update the metrics
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value
        return {m.name: m.result() for m in self.metrics}

    def apply_l2_projection(self, vars_diff):
        projected_diff = tf.clip_by_global_norm(vars_diff, self.delta)[0]
        return [v0 + pv for v0, pv in zip(self.init_vars, projected_diff)]

    def get_config(self):
        return {
            "gamma": self.gamma,
            "delta": self.delta,
            "beta": self.beta,
            "num_networks": self.num_networks,
            "num_layers": self.num_layers,
            "num_neurons": self.num_neurons,
        }


def create_sub_network(n=100, num_neurons=10, num_layers=10, beta=10):
    """
    Creates a submodel with num_layers hidden layers and a truncation layer as
    the last layer.

    Parameters
    ----------
    n: int
        number of training samples
    num_neurons: int, optional
        number of neurons in each hidden layer
    num_layers: int, optional
        number of hidden layers
    beta: float, optional
        parameter for the truncation layer

    Returns
    -------
    model: keras.models.Sequential
        submodel containing a truncation layer as last layer

    """

    # Define submodel
    model = keras.models.Sequential()

    # Create input layer
    model.add(
        Dense(
            units=num_neurons,
            activation="relu",
            kernel_initializer=RandomUniform(minval=-n, maxval=n),
            bias_initializer=RandomUniform(minval=-n, maxval=n),
        )
    )

    # Create num_layers-1 hidden layers
    for _ in range(num_layers - 1):
        model.add(
            Dense(
                units=num_neurons,
                activation="relu",
                kernel_initializer=RandomUniform(minval=-1, maxval=1),
                bias_initializer=RandomUniform(minval=-1, maxval=1),
            )
        )

    # Create output layer
    model.add(
        Dense(
            units=1,
            activation="relu",
            kernel_initializer=RandomUniform(minval=-1, maxval=1),
            bias_initializer=RandomUniform(minval=-1, maxval=1),
        )
    )

    # Create truncation layer
    model.add(TruncateLayer(beta=beta))

    return model


def create_dnn(
    train_shape,
    num_networks=100,
    num_layers=10,
    num_neurons=10,
    beta=10,
    gamma=10,
    delta=1,
):
    """
    Creates a model with num_networks subnetworks with num_layers hidden layers
    each. The output is the average of the outputs of the subnetworks.

    Parameters
    ----------
    train_shape: tuple
        shape of the training data
    num_networks: int, optional
        number of subnetworks to train 
    num_layers: int, optional
        number of hidden layers in each subnetwork 
    num_neurons: int, optional
        number of neurons in each hidden layer 
    beta: float, optional
        parameter for the truncation layer 
    gamma: float, optional
        parameter for the L1 projection layer 
    delta: float, optional
        parameter for the L2 projection

    """
    # Define input shape based on dimension of input variable
    n, d = train_shape
    input_shape = (d,)

    # Create a list containing num_networks DNNs with num_layers hidden layers
    sub_networks = [
        create_sub_network(
            n=n, num_neurons=num_neurons, num_layers=num_layers, beta=beta
        )
        for _ in range(num_networks)
    ]

    # Create the output layer
    output_layer = Dense(
        units=1,
        use_bias=False,
        kernel_initializer=Zeros(),
        kernel_constraint=L1Projection(gamma),
    )

    # Define the structure of the combined model
    inputs = Input(shape=input_shape)
    truncated_outputs = [sub_net(inputs) for sub_net in sub_networks]
    concatenated_outputs = Concatenate()(truncated_outputs)
    outputs = output_layer(concatenated_outputs)

    # Create the model
    model = L2ProjectionModel(
        inputs=inputs,
        outputs=outputs,
        delta=delta,
        sub_networks=sub_networks,
        output_layer=output_layer,
        num_networks=num_networks,
        num_layers=num_layers,
        num_neurons=num_neurons,
        beta=beta,
        gamma=gamma,
    )

    # One-time initialization of initial weights
    model.init_vars  = [tf.identity(v) for v in model.trainable_variables[:-1]]

    return model


In [8]:
# Function to calculate IQR
def calculate_iqr(data):

    q75, q25 = np.percentile(data, [75, 25])
    return q75 - q25


def iqr_median(my_func, x_dim=1, num_samples=10**5, num_repetitions=100):
    """Function to calculate the IQR for a given function"""

    iqr_values = []
    # Generate random samples of x and calculate IQR for each set
    for _ in range(num_repetitions):
        x_samples = np.random.rand(num_samples, x_dim)
        my_func_values = my_func(x_samples)
        iqr_values.append(calculate_iqr(my_func_values))
    # Calculate the median of the IQR values
    median_iqr = np.median(iqr_values)

    return median_iqr


# Generate random data samples
def get_data(my_func, x_dim=1, num_samples=10**3, sigma=0.05):
    """Function to generate random data samples"""

    omega = iqr_median(my_func, x_dim=x_dim)

    x = np.random.rand(num_samples, x_dim)
    y = my_func(x) + sigma * omega * np.random.normal(0, 1, (num_samples,))

    return (x, y)


# Optimized data loading
def preprocess(x, y, batch_size=64, training=False):
    """Function to preprocess data for training"""

    data = tf.data.Dataset.from_tensor_slices((x, y))

    if training:
        data = data.shuffle(buffer_size=len(x))

    data = data.batch(batch_size)
    data = data.prefetch(tf.data.AUTOTUNE)

    return data


In [9]:
tf.test.gpu_device_name()
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
strategy = tf.distribute.MirroredStrategy()

Physical devices cannot be modified after being initialized


In [10]:
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))


Num GPUs Available:  2


In [7]:
def model_builder(hp):
    hp_units = hp.Int("units", min_value=10, max_value=40, step=10)
    hp_layers = hp.Int("layers", min_value=5, max_value=20, step=5)
    hp_nets = hp.Int("networks", min_value=80, max_value=6 * 80, step=80)
    hp_beta = hp.Float('beta', min_value=20, max_value=80)
    hp_gamma = hp.Float('gamma', min_value=30, max_value=80)
    hp_learning_rate = hp.Float(
        "learning_rate", min_value=np.exp(-16), max_value=np.exp(-14), sampling="log"
    )

    # Hier nutzen Sie Ihre angepasste `create_dnn` Funktion mit den hp-Argumenten
    model = create_dnn(
        train_shape=x_train.shape,  # Beispielwerte
        num_networks=hp_nets,
        num_layers=hp_layers,
        num_neurons=hp_units,
        beta=hp_beta,
        gamma=hp_gamma,
        delta=1,
    )

    optimizer = keras.optimizers.SGD(learning_rate=hp_learning_rate)

    model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"]
    )

    return model

In [11]:
def regression_func(x):
    return np.exp(np.linalg.norm(x, axis=1))

input_dim = 7

x, y = get_data(regression_func, x_dim=input_dim, num_samples=100, sigma=0.05)
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.2)

train_data = preprocess(x_train, y_train, batch_size=20*2, training=True)
val_data = preprocess(x_val, y_val, batch_size=20*2, training=False)

In [14]:
tf.debugging.set_log_device_placement(True)

In [19]:
print(tf.test.gpu_device_name())
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)
        
strategy = tf.distribute.MirroredStrategy()

/device:GPU:0
2 Physical GPUs, 2 Logical GPUs


In [12]:
# Check available GPU devices.
print("Available GPU devices: ", tf.config.experimental.list_physical_devices('GPU'))

# Enable device placement logging.
tf.debugging.set_log_device_placement(True)

# Test GPU utilization with a simple operation.
a = tf.constant([[1.0, 2.0, 3.0], [4.0, 5.0, 6.0]])
b = tf.constant([[1.0, 2.0], [3.0, 4.0], [5.0, 6.0]])
c = tf.matmul(a, b)
print(c)


Available GPU devices:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'), PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')]
tf.Tensor(
[[22. 28.]
 [49. 64.]], shape=(2, 2), dtype=float32)


In [18]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():    
    model = create_dnn(
            train_shape=x_train.shape,  # Beispielwerte
            num_networks=160,
            num_layers=10,
            num_neurons=20,
            beta=80,
            gamma=80,
            delta=1,
        )
    optimizer = keras.optimizers.SGD(learning_rate=np.exp(-15))
    model.compile(
        optimizer=optimizer, loss="mean_squared_error", metrics=["mean_squared_error"]
    )
    
model.fit(train_data, epochs = 500)

Epoch 1/500


NotImplementedError: in user code:

    File "/opt/conda/lib/python3.10/site-packages/keras/src/backend/tensorflow/trainer.py", line 105, in one_step_on_data  **
        return self.train_step(data)
    File "/tmp/ipykernel_34/1320630880.py", line 153, in train_step
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
    File "/opt/conda/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py", line 269, in apply_gradients
        self.apply(grads, trainable_variables)
    File "/opt/conda/lib/python3.10/site-packages/keras/src/optimizers/base_optimizer.py", line 334, in apply
        variable.assign(variable.constraint(variable))
    File "/tmp/ipykernel_34/1320630880.py", line 55, in __call__
        return self.apply_l1_projection(w)
    File "/opt/conda/lib/python3.10/site-packages/tensorflow/core/function/polymorphism/function_type.py", line 583, in canonicalize_to_monomorphic
        _make_validated_mono_param(name, arg, poly_parameter.kind,
    Fil

In [15]:
with strategy.scope():
    tuner = kt.Hyperband(
        model_builder,
        objective='val_loss',
        executions_per_trial=2,
        directory='/kaggle/working/',
        project_name='keras-tuner',
        distribution_strategy=strategy
    )

    # Starten des Tuning-Prozesses
    tuner.search(x_train, y_train, epochs=500, validation_data=(x_val, y_val))

# Abrufen der besten Hyperparameter
best_hps = tuner.get_best_hyperparameters(num_trials=1)[0]


Search: Running Trial #1

Value             |Best Value So Far |Hyperparameter
10                |10                |units
10                |10                |layers
480               |480               |networks
21.349            |21.349            |beta
61.206            |61.206            |gamma
1.3542e-07        |1.3542e-07        |learning_rate
2                 |2                 |tuner/epochs
0                 |0                 |tuner/initial_epoch
4                 |4                 |tuner/bracket
0                 |0                 |tuner/round



IndexError: pop from empty list